In [ ]:
# Reference : https://ampl.com/api/extra/python_quickstart.html

## Step 1: Path Setting

In [1]:
from __future__ import print_function
import pandas as pd

In [2]:
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
from amplpy import AMPL, Environment, DataFrame

## Step 2: Create an AMPL object

In [4]:
ampl = AMPL(Environment('C:\\Users\\user\\Desktop\\OR_Kung\\AMPL'))

## Step 3: Select the solver

In [5]:
ampl.setOption('solver','C:\\Users\\user\\Desktop\\OR_Kung\\AMPL\\cplex')  #'gurobi'

## Step 4: Define the model

In [6]:
# ampl.read('FairCHBF.mod')

In [6]:
ampl.eval('''
set I;
set J;

param B {J} >=0;   # benefit of doing the job
param C {J} >=0;   # cost of doing the job
param K {I} >=0;   # capacity of machine 

var X {I,J} >=0 integer;   # Decision var 1
var MinBnft;               # Decision var 2

maximize Benefit: MinBnft;

s.t. Capacity {i in I        }: sum {j in J} C[j] * X[i,j] <= K[i];
s.t. ProdTime {        j in J}: sum {i in I}        X[i,j] <= 1;
s.t. MacBnft  {i in I        }: sum {j in J} B[j] * X[i,j] >= MinBnft;
s.t. Arrange  {i in I, j in J}:                     X[i,j] >= 0;

s.t. MinBenefit : MinBnft >=0;
''')

## Step 5: Define the initial data

In [7]:
job     = ['N1', 'N2', 'N3', 'N4', 'N5']
machine = ['M1', 'M2', 'M3']

# AMPL sets

ampl.getSet('J').setValues(job)
ampl.getSet('I').setValues(machine)

# amplpy DataFrame

ampl.setData(DataFrame(
    index=[('J', job)], 
    columns=[
        ('B', [20,16,12,8,4]), 
        ('C', [10,8,6,4,2])
    ]
))


In [8]:
#  Pandas DataFrame

df = pd.DataFrame({
    'K': [12,12,12]
}, 
    index=machine
)
ampl.setData(DataFrame.fromPandas(df))

## Step 6: Solve the model

In [9]:
ampl.solve()

CPLEX 12.8.0.0: optimal integer solution; objective 20
14 MIP simplex iterations
0 branch-and-bound nodes


## Step 7: Create a Pandas DataFrame with the values of the variable 'Buy'

In [10]:
ampl.getVariable('X').getValues().toPandas()

,X.val
"(M1, N1)",0.0
"(M1, N2)",1.0
"(M1, N3)",0.0
"(M1, N4)",0.0
"(M1, N5)",1.0
"(M2, N1)",0.0
"(M2, N2)",0.0
"(M2, N3)",1.0
"(M2, N4)",1.0
"(M2, N5)",0.0


## Step 8: Display the objective value

In [12]:
MinBnft = ampl.getObjective('Benefit')
print("Objective is:", MinBnft.value())

Objective is: 20.0


## Step 9: Display the values of the decision variable using Bokeh

In [13]:
rows = [tuple(row) for row in X.getValues()]
factors = [index for index, value in rows]
x = [value for index, value in rows]

dot = figure(
    title="Categorical Dot Plot", tools='', toolbar_location=None,
    y_range=factors, x_range=[0,12]
)

dot.segment(0, factors, x, factors, line_width=2, line_color='green')
dot.circle(x, factors, size=15, fill_color='orange', line_color='green', line_width=3)
show(dot)

NameError: name 'X' is not defined

## Other Function

In [22]:
# Increase the costs of beef and ham

cost = ampl.getParameter('cost')
cost.setValues({'BEEF': 5.01, 'HAM': 4.55})
print("Increased costs of beef and ham.")

# 再解一次

ampl.solve()
print("New objective value:", totalcost.value())

# 印出其中項目

Buy = ampl.getVariable('Buy')
print("Buy['BEEF'].val = {}".format(Buy['BEEF'].value()))

# Display the dual value of each constraint

diet = ampl.getConstraint('diet')
for nutr in nutrients:
    print("diet['{}'].dual = {}".format(nutr, diet[nutr].dual()))

Increased costs of beef and ham.
CPLEX 12.8.0.0: optimal solution; objective 144.0120033
0 simplex iterations (0 in phase I)
New objective value: 144.01200332502074
Buy['BEEF'].val = 5.226932668329172
diet['A'].dual = 0.0
diet['C'].dual = 0.0
diet['B1'].dual = 0.0
diet['B2'].dual = 0.7999285120532
diet['NA'].dual = -0.007531172069825434
diet['CAL'].dual = 0.0
